# <font color="blue">線形判別分析（課題）</font>

この課題では，ある旅行会社の顧客データを使用し，「Fisherの線形判別分析」「SVM」「ロジスティック回帰」を用いた予測を行います．

今回のデータセットには，旅行会社の顧客データとして，性別や年収，旅行保険購入の有無などさまざまな変数が含まれています．<br>
ここでの目的変数と説明変数は以下のとおりです．

目的変数<br>
- 旅行保険購入の有無

説明変数（7つ）<br>
- Age：年齢
- GraduateOrNot：大学卒業生かどうか（yes/no）
- AnnualIncome：年収
- FamilyMembers：同居している家族の人数
- ChronicDiseases：慢性疾患（顧客に慢性的な症状があるかどうか）（yes/no）
- FrequentFlyer：顧客が頻繁に航空券を予約するかどうか（yes/no）
- EverTravelledAbroad：顧客が外国に旅行したことがあるかどうか（yes/no）

すなわち，<font color="red">上記の説明変数から「旅行保険購入の有無」を予測する</font>ことを目的とします．<br>
なお，「yes/no」の回答はダミー変数に変換して使用します．<br>

課題のおおまかな流れは以下のとおりです．<br>

1. データのインポートと中身の確認
2. 関係式の作成と評価<br>
   2.1. Fisherの線形判別分析による分類<br>
   2.2. SVMによる分類<br>
   2.3. ロジスティック回帰による分類<br>

## <font color="blue">1. データのインポートと中身の確認</font>

まずは，Google ドライブをマウントし，旅行会社のデータセット（配布ファイルの"5-1_5-2_work.xlsx"）をインポートしてみましょう．<br>
データセットはあらかじめ一部加工したものを配布しています．

<font color="blue">ここは以下のセルをそのまま実行してください．</font><br>

In [ ]:
# Google driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 作業ディレクトリの移動
%cd /content/drive/MyDrive/WASEDADS/05/

Mounted at /content/drive
/content/drive/MyDrive/WASEDADS/05


In [ ]:
#　データをインポートする
# pandasの関数read_excelを使って「5-1_5-2_work.xlsx」を読み込み，dfという変数に代入
import pandas as pd
df = pd.read_excel('/content/drive/My Drive/WASEDADS/05/5-1_5-2_work.xlsx')

`df.head`関数を用いてデータの先頭5行を表示してみましょう．

In [ ]:
# 先頭5行の表示
df.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
3,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0
4,25,Private Sector/Self Employed,No,1150000,4,0,No,No,0


### データセット（各列）の内容

データセットの各列の意味は以下のとおりです．<br>

<font color="blue">青字</font> = 目的変数<br>
<font color="red">赤字</font> = 説明変数<br>

1.<font color="red"> Age：年齢</font><br>
2.<font color="black"> Employment Type：雇用形態<br>
3.<font color="red"> GraduateOrNot：大学卒業生かどうか（yes/no）</font><br>
4.<font color="red"> AnnualIncome：年収（インディアンルピー）</font><br>
5.<font color="red"> FamilyMembers：同居している家族の人数</font><br>
6.<font color="red"> ChronicDiseases：慢性疾患（顧客に慢性的な症状があるかどうか）（1=yes/0=no）</font><br>
7.<font color="red"> FrequentFlyer：顧客が頻繁に航空券を予約するかどうか（yes/no）</font><br>
8.<font color="red"> EverTravelledAbroad：顧客が外国に旅行したことがあるかどうか（yes/no）</font><br>
9.<font color="blue"> TravelInsurance：顧客が旅行保険を購入したかどうか（1=yes/0=no）</font><br>

### データの概要

次に，`df.info`関数を使って，データ各列の情報を取得してみましょう．

In [ ]:
#　データの概要の確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Age                  1500 non-null   int64 
 1   Employment Type      1500 non-null   object
 2   GraduateOrNot        1500 non-null   object
 3   AnnualIncome         1500 non-null   int64 
 4   FamilyMembers        1500 non-null   int64 
 5   ChronicDiseases      1500 non-null   int64 
 6   FrequentFlyer        1500 non-null   object
 7   EverTravelledAbroad  1500 non-null   object
 8   TravelInsurance      1500 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 105.6+ KB


実行結果から，データの型にはint型，object型があることがわかります．<br>
また，<font color="blue">目的変数</font>はすでに0と1で入力してあり，データ型もint型になっていますので，ここでの変換は必要ありません．<br>
Non-Null Countは全体の行数と一致しているので，Null（欠測値）が存在しないこともわかります．<br>

### object型説明変数の変換

説明変数は，「`GraduateOrNot`」，「`FrequentFlyer`」，「`EverTravelledAbroad`」がobject型になっていますので，<br>
`pd.get_dummies`関数を用いてダミー変数に変換しましょう<br>
オプションに`drop_first`を`True`に設定する（`drop_first=True`）ことで，カテゴリの列を1つ削除したダミー変数を作成することができます．<br>
ダミー変数化された説明変数は，自動的に列の後ろに追加されます．<br>

<font color="blue">ここは以下のセルをそのまま実行してください．</font><br>

In [ ]:
# 特定の列に対して変換を実施するには，パラメータのcolumnsを利用します．
#　そのため，まずは，categoryという変数に「GraduateOrNot」，「FrequentFlyer」，「EverTravelledAbroad」を格納します．
#　なお，すべての列を一度に変換するには、columnsを指定する必要はありません．
category = ['GraduateOrNot','FrequentFlyer','EverTravelledAbroad']

#　drop_firstを「True」に設定し，カテゴリの列を1つ削除したダミー変数を作成します．
df_dummies = pd.get_dummies(df, columns=category, drop_first=True)
df_dummies.head()

,Age,Employment Type,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,GraduateOrNot_Yes,FrequentFlyer_Yes,EverTravelledAbroad_Yes
0,31,Government Sector,400000,6,1,0,True,False,False
1,31,Private Sector/Self Employed,1250000,7,0,0,True,False,False
2,28,Private Sector/Self Employed,700000,3,1,0,True,False,False
3,28,Private Sector/Self Employed,700000,8,1,0,True,True,False
4,25,Private Sector/Self Employed,1150000,4,0,0,False,False,False


「`GraduateOrNot`」，「`FrequentFlyer`」，「`EverTravelledAbroad`」の回答が，yes→1，no→0に変換されたことを確認してください<br>
（yes/noの2値であるため，変換後は1列で表示されます）．<br>


### 目的変数のデータ数確認

`df['TravelInsurance']`に対して`value_count()`関数を使用し，目的変数yのデータ数（y=1（yes）とy=0（no））も確認してみましょう．

In [ ]:
# 目的変数yのyesとnoのデータ数をカウント
df['TravelInsurance'].value_counts()

,count
TravelInsurance,
0,800
1,700


`0    800`<br>
`1    700`<br>
と表示されていればOKです．<br>

※noの回答が800，yesの回答が700あります．<br>

## <font color="blue">2. 関係式の作成と評価</font>

関係式（判別境界）の作成手順の大まかな流れは次のようになります．<br>

1. 訓練データとテストデータの準備<br>
関係式の作成に使用する「訓練データ」と「テストデータ」を準備します．ここでは，上記でインポートしたデータを訓練用とテスト用に分割して使用します．<br>
どのような割合で分割するのかはtest_sizeで設定します．ここでは`test_size=0.2`にしているため，説明変数と目的変数を8:2の割合で訓練データとテストデータに分割します（たとえば，0.3にすると，7:3の割合で分割することができます）．<br>
なお，8:2に分割される訓練データとテストデータの組み合わせは分割するたびに変わります．そこで，繰り返し実行しても同じ分割結果になるように，`randam_state`を0に設定してデータを固定します．<br>

2. データの標準化<br>
関係式の作成時（関係式のパラメータの推定）において，説明変数が複数あり，またそれぞれの単位や大きさが異なっている場合，値の小さな変数の影響が小さくなる懸念があります．そこで，そうならないように説明変数の標準化を行ってから，パラメータの推定を行います．標準化を行うと，説明変数の平均値は0，標準偏差は1になります．なお，標準化を行う際に気をつけておくポイントとしては，訓練データの平均値と標準偏差を使用している点です．テストデータは将来手に入るであろう未知データという位置づけのため，そのデータを使って標準化することはできません（scikit-learnのロジスティック回帰では，前回説明した「正則化」が行われる設定になっているので，説明変数の標準化を行ってからパラメータの推定を行います）．

3. 関係式の作成（関係式のパラメータ推定）<br>
scikit-learnのライブラリを利用して，訓練データから関係式のパラメータを推定します．

4. テストデータを用いた関係式の評価<br>
予測結果の精度を評価します．具体的には，テストデータの正解率（予測クラスと正解クラスを比較して一致している割合）で関係式の予測精度を評価します．正解率が高いほど関係式の精度は高いということになります．


### <font color="maroon">2.1. Fisherの線形判別分析による分類</font>

Fisherの線形判別分析を用いた関係式を作成してみましょう．

#### <font color="maroon">ライブラリのインポート</font>

使用するライブラリをまとめてインポートします．

In [ ]:
# Fisherの線形判別分析を用いた関係式作成のためのインポート
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 標準化を行うためのインポート
from sklearn.preprocessing import StandardScaler

# 正解率を計算するためのインポート
from sklearn.metrics import accuracy_score

#### <font color="maroon">訓練データとテストデータの準備</font>

まずは，説明変数，目的変数を`df_dummies`から抽出しましょう．<br>
`df_dummies`に列名を文字列として指定することでその列を抽出できます．<br>

説明変数<br>
- `Age`
- `AnnualIncome`
- `FamilyMembers`
- `ChronicDiseases`
- `GraduateOrNot_Yes`
- `FrequentFlyer_Yes`
- `EverTravelledAbroad_Yes`

目的変数<br>

- `TravelInsurance`

In [ ]:
# 説明変数と目的変数の設定
df_X = df_dummies[['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases', 'GraduateOrNot_Yes', 'FrequentFlyer_Yes', 'EverTravelledAbroad_Yes']]
df_y = df_dummies['TravelInsurance']

`train_test_split`関数を用いて，説明変数と目的変数をそれぞれ訓練データとテストデータに分けましょう．<br>
`train_test_split`関数の引数には，最初に説明変数`df_X`，次に目的変数`df_y`を渡し，<br>
その後に必要なオプション（ここでは`test_size=0.2， random_state=0`）を渡します．<br>

In [ ]:
# 訓練データとテストデータに分ける
# 説明変数と目的変数を8:2の割合で分割するために，test_sizeを0.2に設定
# 繰り返し実行しても同じ分割結果になるように，randam_stateを0に設定
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

`shape`関数（指定したデータフレームの行数・列数を取得）を用いて，分割できているかを確認してみましょう．<br>


In [ ]:
# shape関数を用いて分割できているかを確認
df_X_train.shape, df_X_test.shape, df_y_train.shape, df_y_test.shape

((1200, 7), (300, 7), (1200,), (300,))

データは全部で1500行ありますので，`((1200, 7), (300, 7), (1200,), (300,))`と表示されていればOKです．

#### <font color="maroon">データの標準化</font>

データの標準化を行いましょう．<br>
訓練データの平均値と標準偏差の計算には`fit`関数を使用し，標準化には`transform`関数を使用します．

In [ ]:
# データの標準化を行うための変換器を作成
sc = StandardScaler()

# 説明変数の訓練データの平均値と標準偏差を計算（scに対してfit関数を用いる．引数にはdf_X_trainを渡す）
# 計算結果はscに格納される
sc.fit(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数の訓練データ（df_X_train）を標準化
df_X_train_std = sc.transform(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数のテストデータ（df_X_test）を標準化
df_X_test_std = sc.transform(df_X_test)

#### <font color="maroon">関係式の作成（分析器の準備および訓練データを元にした関係式のパラメータ推定）</font>

まずは，`LinearDiscriminantAnalysis`関数を用いて分析器を準備します．

In [ ]:
# Fisherの線形判別分析を行うための分析器を準備する
clf_fisher = LinearDiscriminantAnalysis()

`clf_fisher`に対して`fit`関数を用いることで，訓練データを元にした関係式のパラメータを推定します．<br>
`fit`関数の引数には訓練データの説明変数`df_X_train_std`と目的変数`df_y_train`を順に渡します．

In [ ]:
# 訓練データを元にした関係式のパラメータを推定（関係式を作成）
clf_fisher.fit(df_X_train_std, df_y_train)

LinearDiscriminantAnalysis()

#### <font color="maroon">テストデータを用いた関係式の評価</font>

作成した関係式（`clf_fisher`）に対して，`predict`関数を用いることで予測クラスを出力してみましょう．<br>
`predict`関数の引数にはテストデータ`df_X2_test_std`を渡します．<br>
また，`print`関数で予測クラスを出力してみましょう．

In [ ]:
# テストデータに対する予測クラスを出力
df_y_test_pred =  clf_fisher.predict(df_X_test_std)
print(df_y_test_pred)

[0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 1 0 1
 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0
 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1
 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0
 0 0 0 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0
 1 0 1 0]


`accuracy_score`関数に予測クラス`df_y_test_pred`と正解クラス`df_y_test`を渡すことで正解率を計算してみましょう．<br>

※<font color="red">引数には，正解クラス`df_y_test`，予測クラス`df_y_test_pred`の順で渡します．</font><br>
※正解率は正解した数をテストデータの数で割ったものです．<br>

In [ ]:
# 予測クラスと正解クラスから正解率を計算
ac_score = accuracy_score(df_y_test, df_y_test_pred)
print('正解率　= %.2f' %(ac_score))

正解率　= 0.73


`正解率　= 0.73`（73%）と表示されていればOKです．

#### <font color="maroon">未知データの予測</font>

作成した関係式を用いて，未知のデータを予測してみましょう．<br>
まずは，未知データのデータフレームを作成します．<br>
各カテゴリには以下の数値を入力しましょう．<br>

- `Age`: `33`
- `AnnualIncome`: `1500000`
- `FamilyMembers`: `4`
- `ChronicDiseases`: `0`
- `GraduateOrNot_Yes`: `1`
- `FrequentFlyer_Yes`: `1`
- `EverTravelledAbroad_Yes`: `1`

In [ ]:
# 未知データの作成（Age, AnnualIncome, FamilyMembers, ChronicDiseases, GraduateOrNot_Yes, FrequentFlyer_Yes, EverTravelledAbroad_Yes）
df_X_newdata = pd.DataFrame(
    data={'Age': [33],
          'AnnualIncome':[1500000],
          'FamilyMembers':[4],
          'ChronicDiseases':[0],
          'GraduateOrNot_Yes':[1],
          'FrequentFlyer_Yes':[1],
          'EverTravelledAbroad_Yes':[1]}
)

説明変数の訓練データの平均値と標準偏差を用いて未知データを標準化します．<br>
標準化には`transform`関数を使用し，引数には未知データを渡します．

In [ ]:
# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて未知データを標準化
df_X_newdata_std = sc.transform(df_X_newdata)

作成した関係式（`clf_fisher`）に対して`predict`関数を用いることで分類の予測クラスを出力します．<br>
また，`clf_fisher`に`predict_proba`関数を用いることで未知データが各クラスに属する確率の推定値を出力します．<br>
`predict`関数と`predict_proba`関数の引数にはテストデータ`df_X_newdata_std`を渡します．<br>
結果は`print`関数を使用して表示させてみましょう．<br>

In [ ]:
print('Fisherの線形判別分析')
print('予測', clf_fisher.predict(df_X_newdata_std)) # 未知データの分類予測クラス
print('確率', clf_fisher.predict_proba(df_X_newdata_std)) # 未知データが各クラスに属する確率の推定値

Fisherの線形判別分析
予測 [1]
確率 [[0.07777024 0.92222976]]


予測の結果，以下のように表示されていればOKです．<br>

`Fisherの線形判別分析`<br>
`予測 [1]`<br>
`確率 [[0.07777024 0.92222976]]`<br>

### <font color="salmon">2.2. SVMによる分類</font>

次に，SVMを用いた関係式を作成してみましょう．<br>
Fisherとは書き方が異なる箇所に★マークを記載しています．<br>

#### <font color="salmon">ライブラリのインポート</font>

使用するライブラリをまとめてインポートします．

In [ ]:
# SVMを用いた関係式作成のためのインポート★
from sklearn.svm import SVC

# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 標準化を行うためのインポート
from sklearn.preprocessing import StandardScaler

# 正解率を計算するためのインポート
from sklearn.metrics import accuracy_score

#### <font color="salmon">訓練データとテストデータの準備</font>

まずは，説明変数，目的変数を`df_dummies`から抽出しましょう．<br>
`df_dummies`に列名を文字列として指定することでその列を抽出できます．<br>

説明変数<br>
- `Age`
- `AnnualIncome`
- `FamilyMembers`
- `ChronicDiseases`
- `GraduateOrNot_Yes`
- `FrequentFlyer_Yes`
- `EverTravelledAbroad_Yes`

目的変数<br>

- `TravelInsurance`

In [ ]:
# 説明変数と目的変数の設定
df_X = df_dummies[['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases', 'GraduateOrNot_Yes', 'FrequentFlyer_Yes', 'EverTravelledAbroad_Yes']]
df_y = df_dummies['TravelInsurance']

`train_test_split`関数を用いて，説明変数と目的変数をそれぞれ訓練データとテストデータに分けましょう．<br>
`train_test_split`関数の引数には，最初に説明変数`df_X`，次に目的変数`df_y`を渡し，<br>
その後に必要なオプション（ここでは`test_size=0.2， random_state=0`）を渡します．<br>

In [ ]:
# 訓練データとテストデータに分ける
# 繰り返し実行しても同じ分割結果になるように，randam_stateを0に設定
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

`shape`関数（指定したデータフレームの行数・列数を取得）を用いて，分割できているかを確認してみましょう．<br>

In [ ]:
# shape関数を用いて分割できているかを確認
df_X_train.shape, df_X_test.shape, df_y_train.shape, df_y_test.shape

((1200, 7), (300, 7), (1200,), (300,))

データは全部で1500行ありますので，`((1200, 7), (300, 7), (1200,), (300,))`と表示されていればOKです．

#### <font color="salmon">データの標準化</font>

データの標準化を行いましょう．<br>
訓練データの平均値と標準偏差の計算には`fit`関数を使用し，標準化には`transform`関数を使用します．<br>

In [ ]:
# データの標準化を行うための変換器を作成
sc = StandardScaler()

# 説明変数の訓練データの平均値と標準偏差を計算（scに対してfit関数を用いる．引数にはdf_X_trainを渡す）
# 計算結果はscに格納される
sc.fit(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数の訓練データ（df_X_train）を標準化
df_X_train_std = sc.transform(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数のテストデータ（df_X_test）を標準化
df_X_test_std = sc.transform(df_X_test)

#### <font color="salmon">関係式の作成（分析器の作成および訓練データを元にした関係式のパラメータ推定）</font>

まずは，`SVC`関数を用いて分析器を準備します．<br>
引数に`kernel = 'linear'`を設定することで，判別境界を線形に指定することができます．<br>

In [ ]:
# SVMを行うための分析器を準備する．　SVC関数の引数に「kernel=‘linear’」を設定して，判別境界を線形に指定する．★
clf_svm = SVC(kernel = 'linear')

次に，`clf_svm`に対して`fit`関数を用いることで，訓練データを元にした関係式のパラメータを推定します．<br>
`fit`関数の引数には訓練データの説明変数`df_X_train_std`と目的変数`df_y_train`を順に渡します．

In [ ]:
# 訓練データを元にした関係式のパラメータを推定（関係式を作成）
clf_svm.fit(df_X_train_std, df_y_train)

SVC(kernel='linear')

#### <font color="salmon">テストデータを用いた関係式の評価</font>

作成した関係式（`clf_svm`）に対して，`predict`関数を用いることで予測クラスを出力してみましょう．<br>
`predict`関数の引数にはテストデータ`df_X_test_std`を渡します．<br>
`print`関数で予測クラスを見てみましょう．<br>

In [ ]:
# テストデータに対する予測クラスを出力
df_y_test_pred = clf_svm.predict(df_X_test_std)
print(df_y_test_pred)

[0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1
 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0
 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0
 1 0 1 0]


`accuracy_score`関数に予測クラス`df_y_test_pred`と正解クラス`df_y_test`を渡すことで正解率を計算してみましょう．<br>

※<font color="red">引数には，正解クラス`df_y_test`，予測クラス`df_y_test_pred`の順で渡します．</font><br>
※正解率は正解した数をテストデータの数で割ったものです．<br>

In [ ]:
# 予測クラスと正解クラスから正解率を計算
ac_score = accuracy_score(df_y_test, df_y_test_pred)
print('正解率　= %.2f' %(ac_score))

正解率　= 0.71


`正解率　= 0.71`（71%）と表示されていればOKです．

#### <font color="salmon">未知データの予測</font>

作成した関係式を用いて，未知のデータを予測してみましょう（未知のデータはFisherの時と同様です）．<br>
まずは，未知データのデータフレームを作成します．<br>
各カテゴリには以下の数値を入力しましょう．<br>

- `Age`: `33`
- `AnnualIncome`: `1500000`
- `FamilyMembers`: `4`
- `ChronicDiseases`: `0`
- `GraduateOrNot_Yes`: `1`
- `FrequentFlyer_Yes`: `1`
- `EverTravelledAbroad_Yes`: `1`

In [ ]:
# 未知データの作成（Age, AnnualIncome, FamilyMembers, ChronicDiseases, GraduateOrNot_Yes, FrequentFlyer_Yes, EverTravelledAbroad_Yes）
df_X_newdata = pd.DataFrame(
    data={'Age': [33],
          'AnnualIncome':[1500000],
          'FamilyMembers':[4],
          'ChronicDiseases':[0],
          'GraduateOrNot_Yes':[1],
          'FrequentFlyer_Yes':[1],
          'EverTravelledAbroad_Yes':[1]}
)

説明変数の訓練データの平均値と標準偏差を用いて未知データを標準化します．<br>
標準化には`transform`関数を使用し，引数には未知データを渡します．

In [ ]:
# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて未知データを標準化
df_X_newdata_std = sc.transform(df_X_newdata)

作成した関係式（`clf_svm`）に対して，`predict`関数を用いることで分類の予測クラスを出力します．<br>
`predict`関数の引数にはテストデータ`df_X_newdata_std`を渡します．<br>
結果は`print`関数を使用して表示させてみましょう．<br>

In [ ]:
print('SVM')
print('予測', clf_svm.predict(df_X_newdata_std)) # 未知データの分類予測クラス

SVM
予測 [1]


予測の結果，以下のように表示されていればOKです．<br>

`SVM`<br>
`予測 [1]`<br>

### <font color="dodgerblue">2.3. ロジスティック回帰による分類</font>

最後に，ロジスティック回帰を用いた関係式を作成してみましょう．<br>
Fisher，SVMとは書き方が異なる箇所に★マークを記載しています．<br>

#### <font color="dodgerblue">ライブラリのインポート</font>

使用するライブラリをまとめてインポートします．

In [ ]:
# ロジスティック回帰の分析器作成のためのインポート★
from sklearn.linear_model import LogisticRegression

# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 標準化を行うためのインポート
from sklearn.preprocessing import StandardScaler

# 正解率を計算するためのインポート
from sklearn.metrics import accuracy_score

#### <font color="dodgerblue">訓練データとテストデータの準備</font>

まずは，説明変数，目的変数を`df_dummies`から抽出しましょう．<br>
`df_dummies`に列名を文字列として指定することでその列を抽出できます．<br>

説明変数<br>
- `Age`
- `AnnualIncome`
- `FamilyMembers`
- `ChronicDiseases`
- `GraduateOrNot_Yes`
- `FrequentFlyer_Yes`
- `EverTravelledAbroad_Yes`

目的変数<br>

- `TravelInsurance`

In [ ]:
# 説明変数と目的変数の設定
df_X = df_dummies[['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases', 'GraduateOrNot_Yes', 'FrequentFlyer_Yes', 'EverTravelledAbroad_Yes']]
df_y = df_dummies['TravelInsurance']

`train_test_split`関数を用いて，説明変数と目的変数をそれぞれ訓練データとテストデータに分けましょう．<br>
`train_test_split`関数の引数には，最初に説明変数`df_X`，次に目的変数`df_y`を渡し，<br>
その後に必要なオプション（ここでは`test_size=0.2， random_state=0`）を渡します．<br>

In [ ]:
# 訓練データとテストデータに分ける
# 説明変数と目的変数を8:2の割合で分割するために，test_sizeを0.2に設定
# 繰り返し実行しても同じ分割結果になるように，randam_stateを0に設定
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

`shape`関数（指定したデータフレームの行数・列数を取得）を用いて，分割できているかを確認してみましょう．<br>

In [ ]:
# shape関数を用いて分割できているかを確認
df_X_train.shape, df_X_test.shape, df_y_train.shape, df_y_test.shape

((1200, 7), (300, 7), (1200,), (300,))

データは全部で1500行ありますので，`((1200, 7), (300, 7), (1200,), (300,))`と表示されていればOKです．

#### <font color="dodgerblue">データの標準化</font>

データの標準化を行いましょう．<br>
訓練データの平均値と標準偏差の計算には`fit`関数を使用し，標準化には`transform`関数を使用します．<br>

In [ ]:
# データの標準化を行うための変換器を作成
sc = StandardScaler()

# 説明変数の訓練データの平均値と標準偏差を計算（scに対してfit関数を用いる．引数にはdf_X_trainを渡す）
# 計算結果はscに格納される
sc.fit(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数の訓練データ（df_X_train）を標準化
df_X_train_std = sc.transform(df_X_train)

# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて説明変数のテストデータ（df_X_test）を標準化
df_X_test_std = sc.transform(df_X_test)

#### <font color="dodgerblue">関係式の作成（分析器の準備および訓練データを元にした関係式のパラメータ推定）</font>

まずは，`LogisticRegression`関数を用いて分析器を準備します．<br>

In [ ]:
# ロジスティック回帰を行うための分析器を準備する★
clf_lr = LogisticRegression()

`clf_lr`に対して`fit`関数を用いることで，訓練データを元にした関係式のパラメータを推定します．<br>
`fit`関数の引数には訓練データの説明変数`df_X_train_std`と目的変数`df_y_train_std`を順に渡します．

In [ ]:
# 訓練データを元にした関係式のパラメータを推定（関係式を作成）
clf_lr.fit(df_X_train_std, df_y_train)

LogisticRegression()

#### <font color="dodgerblue">テストデータを用いた関係式の評価</font>

作成した関係式（`clf_lr`）に対して，`predict`関数を用いることで予測クラスを出力します．<br>
`predict`関数の引数にはテストデータ`df_X_test_std`を渡します．<br>
`print`関数でその予測クラスを見てみましょう．<br>

In [ ]:
# テストデータに対する予測クラスを出力
df_y_test_pred = clf_lr.predict(df_X_test_std)
print(df_y_test_pred)

[0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 0 1
 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0
 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1
 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1
 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0
 0 0 0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0
 1 0 1 0]


`accuracy_score`関数に予測クラス`df_y_test_pred`と正解クラス`df_y_test`を渡すことで正解率を計算します．<br>

※<font color="red">引数には，正解クラス`df_y_test`，予測クラス`df_y_test_pred`の順で渡します．</font><br>

In [ ]:
# 予測クラスと正解クラスから正解率を計算
ac_score = accuracy_score(df_y_test, df_y_test_pred)
print('正解率　= %.2f' %(ac_score))

正解率　= 0.71


`正解率　= 0.71`（71%）と表示されていればOKです．

#### <font color="dodgerblue">未知データの予測</font>

作成した関係式を用いて，未知のデータを予測してみましょう（未知のデータはFisher，SVMの時と同様です）．<br>
まずは，未知データのデータフレームを作成します．<br>
各カテゴリには以下の数値を入力しましょう．<br>

- `Age`: `33`
- `AnnualIncome`: `1500000`
- `FamilyMembers`: `4`
- `ChronicDiseases`: `0`
- `GraduateOrNot_Yes`: `1`
- `FrequentFlyer_Yes`: `1`
- `EverTravelledAbroad_Yes`: `1`

In [ ]:
# 未知データの作成（Age, AnnualIncome, FamilyMembers, ChronicDiseases, GraduateOrNot_Yes, FrequentFlyer_Yes, EverTravelledAbroad_Yes）
df_X_newdata = pd.DataFrame(
    data={'Age': [33],
          'AnnualIncome':[1500000],
          'FamilyMembers':[4],
          'ChronicDiseases':[0],
          'GraduateOrNot_Yes':[1],
          'FrequentFlyer_Yes':[1],
          'EverTravelledAbroad_Yes':[1]}
)

説明変数の訓練データの平均値と標準偏差を用いて未知データを標準化します．<br>
標準化には`transform`関数を使用し，引数には未知データを渡します．

In [ ]:
# scの情報（説明変数の訓練データの平均値と標準偏差）を用いて未知データを標準化
df_X_newdata_std = sc.transform(df_X_newdata)

作成した関係式（`clf_lr`）に対して，`predict`関数を用いることで分類の予測クラスを出力します．<br>
また，`clf_lr`に`predict_proba`関数を用いることで未知データが各クラスに属する確率の推定値を出力します．<br>
`predict`関数と`predict_proba`関数の引数にはテストデータ`df_X_newdata_std`を渡します．<br>
結果は`print`関数を使用して表示させてみましょう．<br>

In [ ]:
print('ロジスティック回帰')
print('予測', clf_lr.predict(df_X_newdata_std)) # 未知データの分類予測クラス
print('確率', clf_lr.predict_proba(df_X_newdata_std)) # 未知データが各クラスに属する確率の推定値

ロジスティック回帰
予測 [1]
確率 [[0.0733246 0.9266754]]
